In [1]:
import pandas as pd
import requests
import numpy as np

The NHL API has no official documentation, but it is somewhat documented at [this repo](https://github.com/Zmalski/NHL-API-Reference)

In [2]:
base_url = 'https://api.nhle.com/stats/rest/en'

## Getting playoff team stats per season

We restrict our timeline to be after the year 2000. We will only include the teams that made the playoffs each season.

In [3]:
def GetTeamStats(start_season=2000, end_season=2025, playoffs=False):

    seasons = [f'{start_season + i}{start_season + i + 1}' for i in range(end_season - start_season)]
    
    cayenneExp = '('
    for season in seasons[:-1]:
        cayenneExp += f'seasonId={season} or '
    cayenneExp += f'seasonId={seasons[-1]})'

    # 2 ~ regular season, 3 ~ playoffs
    cayenneExp += f' and gameTypeId={3 if playoffs else 2}'

    
    url = base_url + f'/team/summary?limit=-1&cayenneExp={cayenneExp}'
    req = requests.get(url)
    
    data = req.json()['data']
    return pd.json_normalize(data)


df = GetTeamStats(playoffs=True)

df = df.sort_values('seasonId')
df.tail()

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,regulationAndOtWins,seasonId,shotsAgainstPerGame,shotsForPerGame,teamFullName,teamId,ties,wins,winsInRegulation,winsInShootout
300,0.498412,5,19,3.80000,12,2.40000,4,None,0.666666,0.666667,...,1,20232024,33.80000,27.20000,New York Islanders,2,None,1,0,0
128,0.521591,19,48,2.52631,52,2.73684,9,None,0.729729,0.675676,...,10,20232024,27.31578,27.94736,Dallas Stars,25,None,10,8,0
203,0.444954,4,15,3.75000,7,1.75000,4,None,0.625000,0.625000,...,0,20232024,26.00000,25.25000,Washington Capitals,15,None,0,0,0
123,0.489864,5,20,4.00000,14,2.80000,4,None,0.846153,0.769231,...,1,20232024,32.00000,27.00000,Tampa Bay Lightning,14,None,1,1,0
142,0.493333,5,22,4.40000,13,2.60000,4,None,0.550000,0.550000,...,1,20232024,31.00000,29.00000,Los Angeles Kings,26,None,1,0,0


We also want to add each playoff team's regular season statistics. We will do this with a merge with the team name and season id as keys.

In [4]:
rs_df = GetTeamStats(playoffs=False)
rs_df.tail()

df = pd.merge(df, rs_df, on=['teamFullName', 'seasonId'], how='inner', suffixes=['', 'RS'])
df = df.drop(['teamIdRS'], axis=1)

There are some factors we want to include that we can derive from this data.

One example is a way to quantify how well a team performed in the playoffs - we can assign a `playoffScore` to each team where
- a team that does not make the playoffs or does not win any games in the playoffs has a score of 0
- a team that does make the playoffs has a score of their wins divided by the total possible wins
- a team that makes the playoffs and wins the cup has a score of 1

In [5]:
df['playoffScore'] = df['wins'] / 16
df.tail()

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,powerPlayNetPctRS,powerPlayPctRS,regulationAndOtWinsRS,shotsAgainstPerGameRS,shotsForPerGameRS,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,playoffScore
371,0.498412,5,19,3.80000,12,2.40000,4,None,0.666666,0.666667,...,0.190476,0.203463,38,32.75609,30.12195,NaN,39,29,1,0.0625
372,0.521591,19,48,2.52631,52,2.73684,9,None,0.729729,0.675676,...,0.225000,0.241666,48,28.65853,31.56097,NaN,52,40,4,0.6250
373,0.444954,4,15,3.75000,7,1.75000,4,None,0.625000,0.625000,...,0.184210,0.206140,36,30.52439,26.48780,NaN,40,32,4,0.0000
374,0.489864,5,20,4.00000,14,2.80000,4,None,0.846153,0.769231,...,0.262096,0.286290,42,29.63414,29.28048,NaN,45,37,3,0.0625
375,0.493333,5,22,4.40000,13,2.60000,4,None,0.550000,0.550000,...,0.197580,0.225806,42,28.00000,31.90243,NaN,44,37,2,0.0625


We also want to be able to quantify how well a team performed in the playoffs in their recent seasons. We will create a column `last3YearsPlayoffScore` that is a team's total playoff score over the last 3 years.

In [6]:
df['last3PlayoffScore'] = 0.0

for index, team in df.iterrows():

    season = int(str(team['seasonId'])[:4])
    last3Seasons = [int(f'{season - 3 + s}{season - 3 + s + 1}') for s in range(3)]

    data = df[df['seasonId'].isin(last3Seasons)]
    data = data[data['teamId'] == team['teamId']]
    last3PlayoffScore = data['playoffScore'].sum()

    df.loc[index, 'last3PlayoffScore'] = last3PlayoffScore
    
df.head(100)

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,powerPlayPctRS,regulationAndOtWinsRS,shotsAgainstPerGameRS,shotsForPerGameRS,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,playoffScore,last3PlayoffScore
0,0.535425,13,30,2.30769,38,2.92307,6,None,0.863636,0.818182,...,0.160857,46,26.76829,26.87804,5.0,46,42,0,0.4375,0.000
1,0.491155,13,34,2.61538,25,1.92307,6,None,0.777777,0.777778,...,0.193460,38,26.34146,27.90243,13.0,38,35,0,0.4375,0.000
2,0.464799,18,44,2.44444,38,2.11111,9,None,0.794520,0.780822,...,0.202666,42,29.23170,29.19512,9.0,42,39,0,0.5625,0.000
3,0.569819,6,14,2.33333,10,1.66666,4,None,0.761904,0.761905,...,0.212464,41,27.71951,28.63414,10.0,41,39,0,0.1250,0.000
4,0.495084,15,34,2.26666,40,2.66666,6,None,0.921875,0.875000,...,0.187012,43,23.57317,30.41463,12.0,43,37,0,0.5625,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.537473,7,13,1.85714,12,1.71428,4,None,0.921052,0.921053,...,0.185011,41,25.40243,28.00000,NaN,50,35,9,0.1875,0.125
96,0.545667,22,41,1.86363,72,3.27272,6,None,0.918367,0.857143,...,0.207161,49,23.53658,34.39024,NaN,54,47,5,1.0000,0.750
97,0.509971,17,54,3.17647,52,3.05882,8,None,0.786666,0.760000,...,0.218181,39,31.81707,28.76829,NaN,42,36,3,0.5625,0.125
98,0.529255,13,32,2.46153,30,2.30769,7,None,0.795454,0.750000,...,0.187165,43,24.15853,29.60975,NaN,49,40,6,0.3750,0.750


We will do the same for the past 5 seasons as well.

In [7]:
df['last5PlayoffScore'] = 0.0

for index, team in df.iterrows():

    season = int(str(team['seasonId'])[:4])
    last5Seasons = [int(f'{season - 5 + s}{season - 5 + s + 1}') for s in range(5)]

    data = df[df['seasonId'].isin(last5Seasons)]
    data = data[data['teamId'] == team['teamId']]
    last5PlayoffScore = data['playoffScore'].sum()

    df.loc[index, 'last5PlayoffScore'] = last5PlayoffScore
    
df.head(100)

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,regulationAndOtWinsRS,shotsAgainstPerGameRS,shotsForPerGameRS,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,playoffScore,last3PlayoffScore,last5PlayoffScore
0,0.535425,13,30,2.30769,38,2.92307,6,None,0.863636,0.818182,...,46,26.76829,26.87804,5.0,46,42,0,0.4375,0.000,0.0000
1,0.491155,13,34,2.61538,25,1.92307,6,None,0.777777,0.777778,...,38,26.34146,27.90243,13.0,38,35,0,0.4375,0.000,0.0000
2,0.464799,18,44,2.44444,38,2.11111,9,None,0.794520,0.780822,...,42,29.23170,29.19512,9.0,42,39,0,0.5625,0.000,0.0000
3,0.569819,6,14,2.33333,10,1.66666,4,None,0.761904,0.761905,...,41,27.71951,28.63414,10.0,41,39,0,0.1250,0.000,0.0000
4,0.495084,15,34,2.26666,40,2.66666,6,None,0.921875,0.875000,...,43,23.57317,30.41463,12.0,43,37,0,0.5625,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.537473,7,13,1.85714,12,1.71428,4,None,0.921052,0.921053,...,41,25.40243,28.00000,NaN,50,35,9,0.1875,0.125,0.5000
96,0.545667,22,41,1.86363,72,3.27272,6,None,0.918367,0.857143,...,49,23.53658,34.39024,NaN,54,47,5,1.0000,0.750,1.1250
97,0.509971,17,54,3.17647,52,3.05882,8,None,0.786666,0.760000,...,39,31.81707,28.76829,NaN,42,36,3,0.5625,0.125,1.1875
98,0.529255,13,32,2.46153,30,2.30769,7,None,0.795454,0.750000,...,43,24.15853,29.60975,NaN,49,40,6,0.3750,0.750,1.3750


We now want to quantify how a team performed in the regular season against 'good' teams - teams above a .500 win percentage.

In [8]:
def GetTeamGamesBySeason(teamId: int, season: int):

    cayenneExp = f'(homeTeamId={teamId} or visitingTeamId={teamId}) and season={season} and gameType=2'
    
    url = base_url + f'/game?cayenneExp={cayenneExp}'
    req = requests.get(url)
    
    data = req.json()['data']
    return pd.json_normalize(data)

# e.g. Florida
GetTeamGamesBySeason(13, 20232024).head()

,id,easternStartTime,gameDate,gameNumber,gameScheduleStateId,gameStateId,gameType,homeScore,homeTeamId,period,season,visitingScore,visitingTeamId
0,2023020014,2023-10-12T20:00:00,2023-10-12,14,1,7,2,2,30,3,20232024,0,13
1,2023020020,2023-10-14T16:00:00,2023-10-14,20,1,7,2,6,52,3,20232024,4,13
2,2023020036,2023-10-16T19:00:00,2023-10-16,36,1,7,2,3,1,3,20232024,4,13
3,2023020052,2023-10-19T19:00:00,2023-10-19,52,1,7,2,3,13,3,20232024,1,10
4,2023020068,2023-10-21T19:00:00,2023-10-21,68,1,7,2,3,13,3,20232024,5,23


In [9]:
def GetTeamsAbove500BySeason(season: int):
    
    cayenneExp = f'seasonId={season}'
    
    url = base_url + f'/team/summary?limit=-1&cayenneExp={cayenneExp}'
    req = requests.get(url)
    
    data = req.json()['data']
    df = pd.json_normalize(data)

    df = df[df['wins'] / df['gamesPlayed'] >= 0.5]
    return df

In [10]:
df['winRateAgainst500Teams'] = 0.0

for index, team in df.iterrows():
    teamGames = GetTeamGamesBySeason(teamId=team['teamId'], season=team['seasonId'])
    above500Teams = GetTeamsAbove500BySeason(team['seasonId'])

    homeCondition = (teamGames['homeTeamId'] == team['teamId']) & (teamGames['visitingTeamId'].isin(above500Teams['teamId']) & (teamGames['homeScore'] > teamGames['visitingScore']))
    awayCondition = (teamGames['visitingTeamId'] == team['teamId']) & (teamGames['homeTeamId'].isin(above500Teams['teamId']) & (teamGames['homeScore'] < teamGames['visitingScore']))
    
    winsAgainst500Teams = len(teamGames[homeCondition | awayCondition])
    gamesPlayed = len(teamGames)
    winRate = winsAgainst500Teams / gamesPlayed

    df.loc[index, 'winRateAgainst500Teams'] = winRate
        
df.head()

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,shotsAgainstPerGameRS,shotsForPerGameRS,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,playoffScore,last3PlayoffScore,last5PlayoffScore,winRateAgainst500Teams
0,0.535425,13,30,2.30769,38,2.92307,6,None,0.863636,0.818182,...,26.76829,26.87804,5.0,46,42,0,0.4375,0.0,0.0,0.121951
1,0.491155,13,34,2.61538,25,1.92307,6,None,0.777777,0.777778,...,26.34146,27.90243,13.0,38,35,0,0.4375,0.0,0.0,0.121951
2,0.464799,18,44,2.44444,38,2.11111,9,None,0.794520,0.780822,...,29.23170,29.19512,9.0,42,39,0,0.5625,0.0,0.0,0.158537
3,0.569819,6,14,2.33333,10,1.66666,4,None,0.761904,0.761905,...,27.71951,28.63414,10.0,41,39,0,0.1250,0.0,0.0,0.134146
4,0.495084,15,34,2.26666,40,2.66666,6,None,0.921875,0.875000,...,23.57317,30.41463,12.0,43,37,0,0.5625,0.0,0.0,0.060976


In [11]:
# save to csv
df.to_csv('NHLTeamPlayoffStats.csv')

We will also get this years (2024-25) team statistics and save it in a separate file

In [12]:
df.head()

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,shotsAgainstPerGameRS,shotsForPerGameRS,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,playoffScore,last3PlayoffScore,last5PlayoffScore,winRateAgainst500Teams
0,0.535425,13,30,2.30769,38,2.92307,6,None,0.863636,0.818182,...,26.76829,26.87804,5.0,46,42,0,0.4375,0.0,0.0,0.121951
1,0.491155,13,34,2.61538,25,1.92307,6,None,0.777777,0.777778,...,26.34146,27.90243,13.0,38,35,0,0.4375,0.0,0.0,0.121951
2,0.464799,18,44,2.44444,38,2.11111,9,None,0.794520,0.780822,...,29.23170,29.19512,9.0,42,39,0,0.5625,0.0,0.0,0.158537
3,0.569819,6,14,2.33333,10,1.66666,4,None,0.761904,0.761905,...,27.71951,28.63414,10.0,41,39,0,0.1250,0.0,0.0,0.134146
4,0.495084,15,34,2.26666,40,2.66666,6,None,0.921875,0.875000,...,23.57317,30.41463,12.0,43,37,0,0.5625,0.0,0.0,0.060976


In [13]:
df_25 = GetTeamStats(start_season=2024, end_season=2026, playoffs=False)
df_25.head()

,faceoffWinPct,gamesPlayed,goalsAgainst,goalsAgainstPerGame,goalsFor,goalsForPerGame,losses,otLosses,penaltyKillNetPct,penaltyKillPct,...,regulationAndOtWins,seasonId,shotsAgainstPerGame,shotsForPerGame,teamFullName,teamId,ties,wins,winsInRegulation,winsInShootout
0,0.538046,73,215,2.94520,238,3.26027,25,4,0.812206,0.784038,...,43,20242025,29.32876,28.28767,Toronto Maple Leafs,10,None,44,35,1
1,0.443474,72,225,3.12500,196,2.72222,32,8,0.745283,0.731133,...,29,20242025,32.08333,27.68055,Anaheim Ducks,24,None,32,23,3
2,0.517369,74,225,3.04054,208,2.81081,29,12,0.825892,0.808036,...,32,20242025,27.71621,28.08108,Utah Hockey Club,59,None,33,25,1
3,0.479775,72,266,3.69444,187,2.59722,43,9,0.768181,0.745455,...,18,20242025,31.50000,27.02777,San Jose Sharks,28,None,20,14,2
4,0.466367,74,206,2.78378,247,3.33783,26,3,0.817204,0.790323,...,43,20242025,25.93243,29.72972,Colorado Avalanche,21,None,45,38,2


In [14]:
for key in df_25.keys():
    if key not in ['seasonId', 'teamFullName', 'teamId']:
        df_25 = df_25.rename(columns={key: f'{key}RS'})

In [15]:
df_25.keys()

Index(['faceoffWinPctRS', 'gamesPlayedRS', 'goalsAgainstRS',
       'goalsAgainstPerGameRS', 'goalsForRS', 'goalsForPerGameRS', 'lossesRS',
       'otLossesRS', 'penaltyKillNetPctRS', 'penaltyKillPctRS', 'pointPctRS',
       'pointsRS', 'powerPlayNetPctRS', 'powerPlayPctRS',
       'regulationAndOtWinsRS', 'seasonId', 'shotsAgainstPerGameRS',
       'shotsForPerGameRS', 'teamFullName', 'teamId', 'tiesRS', 'winsRS',
       'winsInRegulationRS', 'winsInShootoutRS'],
      dtype='object')

In [16]:
df_25['last3PlayoffScore'] = 0.0

for index, team in df_25.iterrows():

    season = int(str(team['seasonId'])[:4])
    last3Seasons = [int(f'{season - 3 + s}{season - 3 + s + 1}') for s in range(3)]

    data = df[df['seasonId'].isin(last3Seasons)]
    data = data[data['teamId'] == team['teamId']]
    last3PlayoffScore = data['playoffScore'].sum()

    df_25.loc[index, 'last3PlayoffScore'] = last3PlayoffScore
    
df_25.head(16)

,faceoffWinPctRS,gamesPlayedRS,goalsAgainstRS,goalsAgainstPerGameRS,goalsForRS,goalsForPerGameRS,lossesRS,otLossesRS,penaltyKillNetPctRS,penaltyKillPctRS,...,seasonId,shotsAgainstPerGameRS,shotsForPerGameRS,teamFullName,teamId,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,last3PlayoffScore
0,0.538046,73,215,2.94520,238,3.26027,25,4,0.812206,0.784038,...,20242025,29.32876,28.28767,Toronto Maple Leafs,10,None,44,35,1,0.6875
1,0.443474,72,225,3.12500,196,2.72222,32,8,0.745283,0.731133,...,20242025,32.08333,27.68055,Anaheim Ducks,24,None,32,23,3,0.0000
2,0.517369,74,225,3.04054,208,2.81081,29,12,0.825892,0.808036,...,20242025,27.71621,28.08108,Utah Hockey Club,59,None,33,25,1,0.0000
3,0.479775,72,266,3.69444,187,2.59722,43,9,0.768181,0.745455,...,20242025,31.50000,27.02777,San Jose Sharks,28,None,20,14,2,0.0000
4,0.466367,74,206,2.78378,247,3.33783,26,3,0.817204,0.790323,...,20242025,25.93243,29.72972,Colorado Avalanche,21,None,45,38,2,1.5625
5,0.517054,74,243,3.28378,193,2.60810,35,9,0.792792,0.765766,...,20242025,28.55405,26.62162,Boston Bruins,6,None,30,23,0,0.7500
6,0.508819,73,243,3.32876,218,2.98630,30,9,0.839130,0.804348,...,20242025,29.00000,26.00000,Montréal Canadiens,8,None,34,25,2,0.0000
7,0.498204,75,197,2.62666,223,2.97333,29,7,0.849514,0.820389,...,20242025,26.42666,28.40000,New Jersey Devils,1,None,39,35,1,0.3125
8,0.480465,73,260,3.56164,236,3.23287,36,6,0.790909,0.759091,...,20242025,28.83561,27.43835,Buffalo Sabres,7,None,31,25,3,0.0000
9,0.477228,72,241,3.34722,228,3.16666,30,9,0.798969,0.752578,...,20242025,29.73611,28.97222,Columbus Blue Jackets,29,None,33,23,6,0.0000


In [17]:
df_25['last5PlayoffScore'] = 0.0

for index, team in df_25.iterrows():

    season = int(str(team['seasonId'])[:4])
    last5Seasons = [int(f'{season - 5 + s}{season - 5 + s + 1}') for s in range(5)]

    data = df[df['seasonId'].isin(last5Seasons)]
    data = data[data['teamId'] == team['teamId']]
    last5PlayoffScore = data['playoffScore'].sum()

    df_25.loc[index, 'last5PlayoffScore'] = last5PlayoffScore
    
df_25.head(16)

,faceoffWinPctRS,gamesPlayedRS,goalsAgainstRS,goalsAgainstPerGameRS,goalsForRS,goalsForPerGameRS,lossesRS,otLossesRS,penaltyKillNetPctRS,penaltyKillPctRS,...,shotsAgainstPerGameRS,shotsForPerGameRS,teamFullName,teamId,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,last3PlayoffScore,last5PlayoffScore
0,0.538046,73,215,2.94520,238,3.26027,25,4,0.812206,0.784038,...,29.32876,28.28767,Toronto Maple Leafs,10,None,44,35,1,0.6875,1.0000
1,0.443474,72,225,3.12500,196,2.72222,32,8,0.745283,0.731133,...,32.08333,27.68055,Anaheim Ducks,24,None,32,23,3,0.0000,0.0000
2,0.517369,74,225,3.04054,208,2.81081,29,12,0.825892,0.808036,...,27.71621,28.08108,Utah Hockey Club,59,None,33,25,1,0.0000,0.0000
3,0.479775,72,266,3.69444,187,2.59722,43,9,0.768181,0.745455,...,31.50000,27.02777,San Jose Sharks,28,None,20,14,2,0.0000,0.0000
4,0.466367,74,206,2.78378,247,3.33783,26,3,0.817204,0.790323,...,25.93243,29.72972,Colorado Avalanche,21,None,45,38,2,1.5625,2.5000
5,0.517054,74,243,3.28378,193,2.60810,35,9,0.792792,0.765766,...,28.55405,26.62162,Boston Bruins,6,None,30,23,0,0.7500,1.4375
6,0.508819,73,243,3.32876,218,2.98630,30,9,0.839130,0.804348,...,29.00000,26.00000,Montréal Canadiens,8,None,34,25,2,0.0000,1.1250
7,0.498204,75,197,2.62666,223,2.97333,29,7,0.849514,0.820389,...,26.42666,28.40000,New Jersey Devils,1,None,39,35,1,0.3125,0.3125
8,0.480465,73,260,3.56164,236,3.23287,36,6,0.790909,0.759091,...,28.83561,27.43835,Buffalo Sabres,7,None,31,25,3,0.0000,0.0000
9,0.477228,72,241,3.34722,228,3.16666,30,9,0.798969,0.752578,...,29.73611,28.97222,Columbus Blue Jackets,29,None,33,23,6,0.0000,0.2500


In [18]:
df_25['winRateAgainst500Teams'] = 0.0

for index, team in df_25.iterrows():
    teamGames = GetTeamGamesBySeason(teamId=team['teamId'], season=team['seasonId'])
    above500Teams = GetTeamsAbove500BySeason(team['seasonId'])

    homeCondition = (teamGames['homeTeamId'] == team['teamId']) & (teamGames['visitingTeamId'].isin(above500Teams['teamId']) & (teamGames['homeScore'] > teamGames['visitingScore']))
    awayCondition = (teamGames['visitingTeamId'] == team['teamId']) & (teamGames['homeTeamId'].isin(above500Teams['teamId']) & (teamGames['homeScore'] < teamGames['visitingScore']))
    
    winsAgainst500Teams = len(teamGames[homeCondition | awayCondition])
    gamesPlayed = len(teamGames)
    winRate = winsAgainst500Teams / gamesPlayed

    df_25.loc[index, 'winRateAgainst500Teams'] = winRate
        
df_25.head()

,faceoffWinPctRS,gamesPlayedRS,goalsAgainstRS,goalsAgainstPerGameRS,goalsForRS,goalsForPerGameRS,lossesRS,otLossesRS,penaltyKillNetPctRS,penaltyKillPctRS,...,shotsForPerGameRS,teamFullName,teamId,tiesRS,winsRS,winsInRegulationRS,winsInShootoutRS,last3PlayoffScore,last5PlayoffScore,winRateAgainst500Teams
0,0.538046,73,215,2.94520,238,3.26027,25,4,0.812206,0.784038,...,28.28767,Toronto Maple Leafs,10,None,44,35,1,0.6875,1.0,0.195122
1,0.443474,72,225,3.12500,196,2.72222,32,8,0.745283,0.731133,...,27.68055,Anaheim Ducks,24,None,32,23,3,0.0000,0.0,0.134146
2,0.517369,74,225,3.04054,208,2.81081,29,12,0.825892,0.808036,...,28.08108,Utah Hockey Club,59,None,33,25,1,0.0000,0.0,0.134146
3,0.479775,72,266,3.69444,187,2.59722,43,9,0.768181,0.745455,...,27.02777,San Jose Sharks,28,None,20,14,2,0.0000,0.0,0.109756
4,0.466367,74,206,2.78378,247,3.33783,26,3,0.817204,0.790323,...,29.72972,Colorado Avalanche,21,None,45,38,2,1.5625,2.5,0.231707


In [19]:
df_25.to_csv('2025TeamStats.csv')